In [1]:
import pandas as pd
from data_retrievel_and_feature_extraction import uniprot_info as uni


In [4]:
##### Run this cell next after finishing with REVEL extraction.

# Read the file
am_df = pd.read_csv('C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\AlphaMissense\\AlphaMissense_aa_substitutions.tsv', sep='\t')
# Read the predictions file
predictions = pd.read_csv('C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\predictions_of_other_tools_for_all_dvd_variants.csv')

# For each row in the predictions file, get the alpha missense prediction from the am_df
for index, row in predictions.iterrows():
    uniprot_id = row['uniprot_id']
    variant = row['variant']
    # get the pathogenicity prediction of alphamissense for the variant
    am_class = am_df[(am_df['uniprot_id'] == uniprot_id) & (am_df['protein_variant'] == variant)]['am_class'].values
    am_pathogenicity = am_df[(am_df['uniprot_id'] == uniprot_id) & (am_df['protein_variant'] == variant)]['am_pathogenicity'].values
    # Add the score and prediction to the predictions file
    predictions.loc[index, 'am_class'] = am_class
    predictions.loc[index, 'am_pathogenicity'] = am_pathogenicity
    
# print the predictions file with the added columns
predictions
    


KeyboardInterrupt



In [29]:
protein_type = "collagens_proteins"
genes_df = pd.read_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\Data\\{protein_type}_data.csv")
genes_list = genes_df["gene"].unique().tolist()
genes_list

['COL11A1',
 'COL4A3',
 'COL9A3',
 'COL11A2',
 'COL4A5',
 'COL2A1',
 'COL4A4',
 'COL9A1',
 'COL9A2',
 'COL4A6']

In [31]:
# Get AlphaMissense predictions for all genes in the list
import csv
from data_retrievel_and_feature_extraction import uniprot_info as uni


# In the next two parts, it's either running the first part to get the gene_uniprot_dict, or the second part to read it from an existing csv file.

## Create a dictionary of gene names and uniprot ids, when doesn't exist
gene_uniprot_dict = {}
for gene in genes_list:
    uniprot_id = uni.get_uniprot_id(gene)
    gene_uniprot_dict[gene] = uniprot_id
# Create a csv file with the gene names and uniprot ids
with open(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\Data\\gene_uniprot_list_{protein_type}.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["gene", "uniprot_id"])
    for key, value in gene_uniprot_dict.items():
        writer.writerow([key, value])

# get gene_uniprot_dict from the csv file, if already exists
# gene_uniprot_dict = {}
# with open(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\Data\\gene_uniprot_list_{protein_type}.csv", 'r') as file:
#     reader = csv.reader(file)
#     next(reader)
#     for row in reader:
#         gene_uniprot_dict[row[0]] = row[1]

In [33]:
# Read the file and remove genes that are not in the list
tsv_file_path = 'C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\AlphaMissense\\AlphaMissense_aa_substitutions.tsv'

uniprot_ids = gene_uniprot_dict.values()

# convert the tsv file to dataframe
am_df = pd.read_csv(tsv_file_path, sep='\t')
# write a new dataframe with only the genes in the uniprot_ids list
am_df = am_df[am_df['uniprot_id'].isin(uniprot_ids)]
am_df
    

,uniprot_id,protein_variant,am_pathogenicity,am_class
32295346,P02458,M1A,0.1694,benign
32295347,P02458,M1C,0.1616,benign
32295348,P02458,M1D,0.6195,pathogenic
32295349,P02458,M1E,0.3755,ambiguous
32295350,P02458,M1F,0.1309,benign
...,...,...,...,...
79238071,Q14055,P689S,0.1625,benign
79238072,Q14055,P689T,0.1523,benign
79238073,Q14055,P689V,0.2259,benign
79238074,Q14055,P689W,0.6077,pathogenic


In [34]:
# Read the predictions file
if protein_type == "transmembrane_proteins":
    protein_type = "transmembrane_residues"
my_prediction = pd.read_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\{protein_type}\\predictions_vs_real_{protein_type}.csv")
my_prediction

,gene,variant,prediction,reality
0,COL11A1,G1249V,1,1
1,COL11A1,G1516V,1,1
2,COL11A1,R1733C,1,1
3,COL11A1,G1800C,1,1
4,COL11A1,R1076C,1,1
...,...,...,...,...
1091,COL9A2,G30R,1,0
1092,COL9A2,A336E,1,0
1093,COL9A2,L335V,0,0
1094,COL9A2,R32Q,0,0


In [35]:
# Change column names to match the other file
am_predictions = am_df.rename(columns={"protein_variant": "variant"})

# Add gene column to the am_predictions
am_predictions["gene"] = am_predictions["uniprot_id"].map({v: k for k, v in gene_uniprot_dict.items()})

In [36]:
# Join the information from am_predictions to my_prediction by gene and variant
merged = pd.merge(my_prediction, am_predictions, on=["gene", "variant"])
merged

,gene,variant,prediction,reality,uniprot_id,am_pathogenicity,am_class
0,COL11A1,G1249V,1,1,P12107,0.9505,pathogenic
1,COL11A1,G1516V,1,1,P12107,0.9748,pathogenic
2,COL11A1,R1733C,1,1,P12107,0.7065,pathogenic
3,COL11A1,G1800C,1,1,P12107,0.9094,pathogenic
4,COL11A1,R1076C,1,1,P12107,0.8094,pathogenic
...,...,...,...,...,...,...,...
1091,COL9A2,G30R,1,0,Q14055,0.7010,pathogenic
1092,COL9A2,A336E,1,0,Q14055,0.2113,benign
1093,COL9A2,L335V,0,0,Q14055,0.0859,benign
1094,COL9A2,R32Q,0,0,Q14055,0.0769,benign


In [37]:
# Delete all rows with ambiguous in AF score
merged = merged[~(merged['am_class'] == "ambiguous")]
merged

,gene,variant,prediction,reality,uniprot_id,am_pathogenicity,am_class
0,COL11A1,G1249V,1,1,P12107,0.9505,pathogenic
1,COL11A1,G1516V,1,1,P12107,0.9748,pathogenic
2,COL11A1,R1733C,1,1,P12107,0.7065,pathogenic
3,COL11A1,G1800C,1,1,P12107,0.9094,pathogenic
4,COL11A1,R1076C,1,1,P12107,0.8094,pathogenic
...,...,...,...,...,...,...,...
1091,COL9A2,G30R,1,0,Q14055,0.7010,pathogenic
1092,COL9A2,A336E,1,0,Q14055,0.2113,benign
1093,COL9A2,L335V,0,0,Q14055,0.0859,benign
1094,COL9A2,R32Q,0,0,Q14055,0.0769,benign


In [38]:
# Make "benign" and "pathogenic" to 0 and 1
merged.loc[merged['am_class'] == "benign", 'am_class'] = 0
merged.loc[merged['am_class'] == "pathogenic", 'am_class'] = 1

In [39]:
# print how many pathogenic and benign variants are in the EVE file
print(f"Number of pathogenic variants in am: {len(merged[merged['am_class'] == 1])}")
print(f"Number of benign variants in am: {len(merged[merged['am_class'] == 0])}")

Number of pathogenic variants in am: 835
Number of benign variants in am: 200


In [40]:
merged.to_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\protein_type_predictors\\alphamissense_on_my_data_predictions_{protein_type}.csv")